In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,23991
2,Huelva,Confirmados PDIA 14 días,2308
3,Huelva,Tasa PDIA 14 días,"449,7534929945242"
4,Huelva,Confirmados PDIA 7 días,681
5,Huelva,Total Confirmados,24221
6,Huelva,Curados,13426
7,Huelva,Fallecidos,250


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  23991.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6391.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 654 personas en los últimos 7 días 

Un positivo PCR cada 246 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,23991.0,681.0,2308.0,513170.0,132.704562,449.753493,209.0
Huelva-Costa,14608.0,450.0,1418.0,289548.0,155.414646,489.728819,131.0
Huelva (capital),6391.0,220.0,584.0,143837.0,152.950910,406.015142,75.0
Condado-Campiña,6633.0,177.0,682.0,156231.0,113.293777,436.533082,60.0
Lepe,1943.0,67.0,208.0,27880.0,240.315638,746.054519,19.0
Moguer,791.0,33.0,84.0,21867.0,150.912334,384.140486,16.0
Almonte,789.0,46.0,108.0,24507.0,187.701473,440.690415,13.0
Bollullos Par del Condado,568.0,21.0,90.0,14387.0,145.965107,625.564746,12.0
Cartaya,874.0,23.0,86.0,20083.0,114.524722,428.222875,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,163.0,10.0,46.0,2885.0,346.620451,1594.454073,1.0
Paymogo,71.0,1.0,11.0,1162.0,86.058520,946.643718,-1.0
Villalba del Alcor,437.0,1.0,27.0,3366.0,29.708853,802.139037,1.0
Isla Cristina,1959.0,48.0,169.0,21393.0,224.372458,789.978030,9.0
Lepe,1943.0,67.0,208.0,27880.0,240.315638,746.054519,19.0
Ayamonte,1169.0,33.0,147.0,21104.0,156.368461,696.550417,5.0
Bollullos Par del Condado,568.0,21.0,90.0,14387.0,145.965107,625.564746,12.0
Palma del Condado (La),961.0,13.0,66.0,10801.0,120.359226,611.054532,2.0
Villanueva de los Castillejos,143.0,5.0,17.0,2825.0,176.991150,601.769912,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villalba del Alcor,437.0,1.0,27.0,3366.0,29.708853,802.139037,1.0,0.037037
San Bartolomé de la Torre,189.0,1.0,15.0,3761.0,26.588673,398.830098,1.0,0.066667
San Juan del Puerto,351.0,3.0,34.0,9411.0,31.877590,361.279354,1.0,0.088235
Paymogo,71.0,1.0,11.0,1162.0,86.058520,946.643718,-1.0,0.090909
Niebla,158.0,1.0,9.0,4116.0,24.295432,218.658892,0.0,0.111111
Almonaster la Real,56.0,1.0,7.0,1788.0,55.928412,391.498881,1.0,0.142857
Rociana del Condado,293.0,6.0,41.0,7939.0,75.576269,516.437839,5.0,0.146341
Trigueros,222.0,4.0,25.0,7862.0,50.877639,317.985245,0.0,0.160000
Beas,158.0,4.0,23.0,4341.0,92.144667,529.831836,1.0,0.173913
